In [1]:
# import 
""" json.dumps(): 将 Python 对象编码成 JSON 字符串。 json.dump(): 将 Python 对象编码成 JSON 字符串，并写入到文件中。json.loads(): 将已编码的 JSON 字符串解码为 Python 对象。json.load(): 读取文件中的 JSON 编码的数据，并解码为 Python 对象。
    """
import os
import json
import torch
torch.cuda.is_available()
def json_standard(input_object):
    return json.dumps(input_object,ensure_ascii=False,indent=4)
def read_file(file_path:str):
    with open(file_path,"r",encoding="utf-8") as i_f:
        return json.load(i_f)
def write_file(file_path:str,obj,overwrite=False):
    if os.path.exists(file_path)==False or overwrite==True:
        with open(file_path,"w",encoding="utf-8") as o_f:
            json.dump(obj,o_f,ensure_ascii=False)
        print(f"写入{file_path}")
    else:
        print(f"{file_path}文件已经存在")
def check_file(file_path:str):
    """返回该路径下是否存在文件"""
    is_exist=os.path.exists(file_path)
    if is_exist:
        file_name=file_path.split("/")[-1]
        print(f"{file_name} exist")
    return is_exist
import torch
import json
from transformers import (AutoTokenizer)
model_id="/home/ckqsudo/code2024/0model/Mistral-7B-Instruct-v0.2"
tokenizer=AutoTokenizer.from_pretrained(model_id)
# 这里不设置 trust_remote_code也行
tokenizer.padding_size='right'#填充符的位置

In [2]:
en_processed_data=read_file("/home/ckqsudo/code2024/Mistral-7b-finetune/data/en_regular_cache4.json")

In [3]:
len(en_processed_data)

563

In [6]:
en_processed_data[0].keys()

dict_keys(['theme', 'is_same_qa', 'is_same_case', 'background', 'pre_reasoning', 'topic_dialog', 'post_reasoning'])

In [5]:
a=[1,2,5]
a.extend([7])
a

[1, 2, 5, 7]

In [10]:
def filter_one_participant(cur_dataset):
    new_dataset=[]
    for topic_dialog in cur_dataset:
        participant_set=set()
        for dialog_item in topic_dialog["topic_dialog"]["dialog"]:
            participant_set.add(dialog_item["participant"])
        if len(participant_set)==1:
            new_dataset[-1]["topic_dialog"]["dialog"].extend(topic_dialog["topic_dialog"]["dialog"])
            new_dataset[-1]["post_reasoning"]+=topic_dialog["pre_reasoning"]
        else:
            new_dataset.append(topic_dialog)
    return new_dataset
def regular_data(cur_data):
    for x,item in enumerate(cur_data):
        for y,dialog_item in enumerate(item["topic_dialog"]["dialog"]):
            text=dialog_item["text"]
            while text.startswith(" "):
                text=text[1:]
            while text.endswith(" "):
                text=text[:-1]
            cur_data[x]["topic_dialog"]["dialog"][y]["text"]=text

def filter_reasoning_observation(cur_data):
    for x,topic_dialog in enumerate(cur_data):
        for y,dialog_item in enumerate(topic_dialog["topic_dialog"]["dialog"]):
            if "Therapist" in dialog_item["participant"]:
                if 'reasoning' not in dialog_item.keys():
                    
                    cur_data[x]["topic_dialog"]["dialog"][y]['reasoning']=""
            else:
                if 'observation' not in dialog_item.keys():
                    cur_data[x]["topic_dialog"]["dialog"][y]['observation']=""
    
cache_5_dataset=filter_one_participant(en_processed_data)
regular_data(cache_5_dataset)
filter_reasoning_observation(cache_5_dataset)

In [11]:
for x,topic_dialog in enumerate(cache_5_dataset):
    for y,dialog_item in enumerate(topic_dialog["topic_dialog"]['dialog']):
        if "Therapist" in dialog_item["participant"]:
            if 'reasoning' not in dialog_item.keys():
                print(dialog_item)

In [12]:
len(en_processed_data),len(cache_5_dataset)

(563, 520)

In [13]:
for topic_dialog in cache_5_dataset:
    participant_set=set()
    for dialog_item in topic_dialog["topic_dialog"]["dialog"]:
        participant_set.add(dialog_item["participant"])
    if len(participant_set)==1:
        print(participant_set)

In [ ]:
# def regular_data(cur_data): #这个也应该在之前做
#     for x,item in enumerate(cur_data):
#         for y,dialog_item in enumerate(item["topic_dialog"]["dialog"]):
#             text=dialog_item["text"]
#             while text.startswith(" "):
#                 text=text[1:]
#             while text.endswith(" "):
#                 text=text[:-1]
#             cur_data[x]["topic_dialog"]["dialog"][y]["text"]=text
# regular_data(en_data)

In [14]:
write_file("en_regular_cache5.json",cache_5_dataset,True)

写入en_regular_cache5.json
